# Capstone: Automatisch waterdesign voor Tripel / IPA / Stout

**Doel.** In dit capstone-notebook ontwerp je **volledig automatisch** een brouwwaterplan voor drie klassieke bierstijlen (**Tripel**, **IPA**, **Stout**) op basis van:

1) je **bronwater** (tap/RO/gebotteld)  
2) een **stijldoelprofiel** (Ca, Mg, Na, Cl, SO₄, HCO₃)  
3) **zouttoevoegingen** (CaCl₂·2H₂O, CaSO₄·2H₂O, MgSO₄·7H₂O, NaCl, NaHCO₃)  
4) **zuurcorrectie** (melkzuur of fosforzuur) om een **mash pH-doel** te benaderen

> Dit is een **didactisch** model (orde-grootte), niet een volledige waterchemie-engine. Het is bedoeld om inzicht te geven in **trade-offs** en **optimalisatie**.

---

## Workflow (1 pagina)

1. Kies stijl → doelprofiel + pH-doel  
2. Kies volume (brouw-/maischwater)  
3. Kies bronwaterbronnen (bv. Tap + RO)  
4. Optimaliseer:
   - mengverhouding water
   - zoutgrammen per batch
   - zuurvolume
5. Controleer output en interpretatievragen


## 1. Theorie (kort)

### 1.1 Ionenbalans en eenheden
Waterprofielen worden typisch opgegeven in **mg/L** voor ionen zoals Ca²⁺, Mg²⁺, Na⁺, Cl⁻, SO₄²⁻, HCO₃⁻.  
Als je een zout toevoegt (in gram), verhoog je de ionconcentraties volgens de **massa-fractie** en het batchvolume.

### 1.2 Residual Alkalinity (RA)
Een veelgebruikte benadering is:

\[
RA\;[\mathrm{mg/L\;as\;CaCO_3}] = Alkalinity\;[\mathrm{mg/L\;as\;CaCO_3}] - 0.714\,Ca - 0.585\,Mg
\]

met Ca en Mg in **mg/L**.  
Alkalinity wordt vaak benaderd via bicarbonaat:

\[
Alkalinity\;[\mathrm{mg/L\;as\;CaCO_3}] \approx 50 \cdot \frac{HCO_3\;[\mathrm{mg/L}]}{61}
\]

### 1.3 pH-doel (vereenvoudigd)
We koppelen RA aan een **pH-shift** via een eenvoudige lineaire gevoeligheid:
\[
\Delta pH \approx k_{RA}\cdot \frac{RA}{100}
\]
en corrigeren met zuur via een equivalente **neutralisatie** van alkaliniteit:
\[
\Delta Alk \propto \text{mmol H}^+ / L
\]

De parameters (k's) zijn didactisch en kunnen gekalibreerd worden op basis van metingen.


In [ ]:
# --- Imports ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## 2. Parameters

Vul hier je **brouwvolume** en **waterbronnen** in.  
Standaard staan er voorbeeldwaarden (realistische orde van grootte).

- *Tap* = lokaal kraanwater (voorbeeld)  
- *RO* = omgekeerde osmose (≈ 0 mg/L alles)  
- *Bottled* = optioneel (bv. Spa Reine is laag-mineraal; pas aan naar jouw etiket)


In [ ]:
# --- Batch / volumes ---
V_total_L = 30.0      # [L] totaal brouwwater dat je ontwerpt (mash + sparge samen)
V_mash_L  = 18.0      # [L] deel dat naar de maisch gaat (voor pH-model)
pH_target = 5.30      # [-] mash pH doel (bij 20°C meting, typisch 5.2–5.5)

# --- Waterbronnen (mg/L) ---
# Zet op basis van jouw wateranalyse (of brouwwater-rapport).
water_sources = {
    "Tap":     {"Ca": 55, "Mg": 8,  "Na": 20, "Cl": 35, "SO4": 45,  "HCO3": 160},
    "RO":      {"Ca": 0,  "Mg": 0,  "Na": 0,  "Cl": 0,  "SO4": 0,   "HCO3": 0},
    "Bottled": {"Ca": 10, "Mg": 2,  "Na": 5,  "Cl": 5,  "SO4": 8,   "HCO3": 25},  # optioneel
}

# Welke bronnen wil je toelaten in blending?
allowed_sources = ["Tap", "RO"]   # bv. ["Tap","RO"] of ["Tap","RO","Bottled"]

# --- Zouten die we toelaten (gram per batch, grenzen) ---
# Grenzen zijn didactisch; pas aan naar jouw praktijk.
salt_bounds_g = {
    "CaCl2_2H2O": (0.0, 12.0),  # calciumchloride dihydraat
    "CaSO4_2H2O": (0.0, 20.0),  # gips
    "MgSO4_7H2O": (0.0, 10.0),  # epsom
    "NaCl":       (0.0, 8.0),
    "NaHCO3":     (0.0, 10.0),  # baksoda (voor donkere bieren)
}

# --- Zuur (melkzuur 88% of fosforzuur 10%) ---
acid_type = "lactic_88"   # "lactic_88" of "phosphoric_10"
acid_bounds_mL = (0.0, 25.0)

# --- Modelparameters (kalibreerbaar) ---
k_RA_to_pH = 0.12   # pH shift per (RA/100 mg/L as CaCO3)  (orde-grootte)
pH_base_malt = 5.55 # basis mash pH zonder water-effecten (vereenvoudigd)


## 3. Stijldoelprofielen

Onderstaande targets zijn **didactisch** en representeren typische richtingen:

- **Tripel**: relatief “zacht”, chloride licht hoger dan sulfaat, lage bicarbonaat  
- **IPA**: hogere sulfaat voor hopbitterheid/“crispness”, matig chloride  
- **Stout**: hogere bicarbonaat/alkaliniteit om donkere moutzuurheid te bufferen, chloride iets hoger

Je kan deze doelwaarden aanpassen of uitbreiden.


In [ ]:
style_profiles = {
    "Tripel": {"Ca": 60, "Mg": 10, "Na": 15, "Cl": 70,  "SO4": 60,  "HCO3": 30,  "pH_target": 5.30},
    "IPA":    {"Ca": 90, "Mg": 10, "Na": 20, "Cl": 50,  "SO4": 180, "HCO3": 30,  "pH_target": 5.25},
    "Stout":  {"Ca": 80, "Mg": 10, "Na": 30, "Cl": 90,  "SO4": 80,  "HCO3": 180, "pH_target": 5.40},
}

style = "Tripel"   # kies: "Tripel", "IPA", "Stout"
target = style_profiles[style].copy()
pH_target = target.pop("pH_target")

target


In [ ]:
# --- Chemie-hulpfuncties (mg/L from grams added) ---

MOLAR_MASS = {
    "CaCl2_2H2O": 147.014,  # g/mol
    "CaSO4_2H2O": 172.171,
    "MgSO4_7H2O": 246.474,
    "NaCl": 58.443,
    "NaHCO3": 84.007,
}

# Ion molar masses
MM = {"Ca":40.078, "Mg":24.305, "Na":22.990, "Cl":35.453, "SO4":96.06, "HCO3":61.016}

# Stoichiometry: moles ion per mole salt
STOICH = {
    "CaCl2_2H2O": {"Ca":1, "Cl":2},
    "CaSO4_2H2O": {"Ca":1, "SO4":1},
    "MgSO4_7H2O": {"Mg":1, "SO4":1},
    "NaCl":       {"Na":1, "Cl":1},
    "NaHCO3":     {"Na":1, "HCO3":1},
}

def blend_sources(fractions, sources_dict, keys=("Ca","Mg","Na","Cl","SO4","HCO3")):
    """Lineaire blending in mg/L. fractions som=1."""
    out = {k:0.0 for k in keys}
    for frac, name in zip(fractions, sources_dict.keys()):
        for k in keys:
            out[k] += frac * sources_dict[name][k]
    return out

def add_salts(profile_mgL, salts_g, V_L, keys=("Ca","Mg","Na","Cl","SO4","HCO3")):
    """Voegt zouten toe: returns new mg/L profile."""
    out = dict(profile_mgL)
    for salt, g in salts_g.items():
        if g <= 0: 
            continue
        mol = g / MOLAR_MASS[salt]
        for ion, nu in STOICH[salt].items():
            mg_added = mol * nu * MM[ion] * 1000.0  # mg total in batch
            out[ion] += mg_added / V_L
    return out

def alkalinity_as_CaCO3_from_HCO3(HCO3_mgL):
    return 50.0 * (HCO3_mgL / 61.0)

def residual_alkalinity_CaCO3(profile):
    Alk = alkalinity_as_CaCO3_from_HCO3(profile["HCO3"])
    return Alk - 0.714*profile["Ca"] - 0.585*profile["Mg"]

def acid_equivalents_mmol_per_mL(acid_type):
    # grove equivalenten: lactic 88% ~ 11.7 mmol H+/mL (pKa genegeerd; als sterke zuurbron benaderd)
    # phosphoric 10% ~ 1.0 mmol H+/mL (zeer grof, didactisch)
    if acid_type == "lactic_88":
        return 11.7
    if acid_type == "phosphoric_10":
        return 1.0
    raise ValueError("unknown acid_type")

def apply_acid_to_HCO3(profile, acid_mL, V_L, acid_type):
    """Neutraliseert bicarbonaat als alkaliniteit: HCO3 mg/L omlaag (niet onder 0)."""
    eq = acid_equivalents_mmol_per_mL(acid_type)  # mmol/mL
    mmol_total = acid_mL * eq
    mmol_per_L = mmol_total / V_L
    # 1 mmol/L alkaliniteit ~ 50 mg/L as CaCO3; bicarbonaat 61 mg per mmol
    HCO3_reduction_mgL = mmol_per_L * 61.0
    out = dict(profile)
    out["HCO3"] = max(0.0, out["HCO3"] - HCO3_reduction_mgL)
    return out

def predict_mash_pH(profile_mgL_mash, pH_base_malt=5.55, k_RA_to_pH=0.12):
    RA = residual_alkalinity_CaCO3(profile_mgL_mash)
    return pH_base_malt + k_RA_to_pH*(RA/100.0)

print("OK")


## 4. Optimalisatie

We minimaliseren een **kostenfunctie** die de afwijking t.o.v. het doelprofiel en pH-doel straft.

- Ion-targets: gewogen kwadratische fout (mg/L)  
- pH-target: gewogen kwadratische fout  
- Optioneel: zachte straf op “te veel zout” (praktische eenvoud)

Omdat we in een onderwijssetting blijven, gebruiken we een **robuste grid/random search** zonder externe optimizers.


In [ ]:
# --- Optimalisatie-instellingen ---
np.random.seed(0)

N_trials = 20000   # verhoog voor beter resultaat (kan trager)
w_ions = {"Ca":1.0, "Mg":0.5, "Na":0.5, "Cl":1.0, "SO4":1.0, "HCO3":1.0}
w_pH = 250.0       # pH is gevoelig: hogere weging
w_salt = 0.02      # penaliseer grote hoeveelheden zout (g^2)

keys = ["Ca","Mg","Na","Cl","SO4","HCO3"]

# Selecteer bronnen
sources_sel = {k: water_sources[k] for k in allowed_sources}
source_names = list(sources_sel.keys())
n_sources = len(source_names)

def random_simplex(n):
    x = np.random.rand(n)
    x /= x.sum()
    return x

def sample_uniform(bounds):
    lo, hi = bounds
    return lo + (hi-lo)*np.random.rand()

def score(profile, target, pH_pred, pH_target, salts_g):
    s = 0.0
    for ion in keys:
        s += w_ions[ion]*(profile[ion]-target[ion])**2
    s += w_pH*(pH_pred - pH_target)**2
    s += w_salt*sum((g**2 for g in salts_g.values()))
    return s

best = None

for _ in range(N_trials):
    # 1) blend
    frac = random_simplex(n_sources)
    blended = blend_sources(frac, {name:sources_sel[name] for name in source_names})

    # 2) salts
    salts_g = {k: sample_uniform(v) for k,v in salt_bounds_g.items()}
    salted = add_salts(blended, salts_g, V_total_L)

    # 3) acid
    acid_mL = sample_uniform(acid_bounds_mL)
    acid_profile = apply_acid_to_HCO3(salted, acid_mL, V_total_L, acid_type)

    # 4) mash pH (we nemen aan dat mashwater dezelfde ionen heeft als totale batch; voor nauwkeuriger: separaat mash-water design)
    pH_pred = predict_mash_pH(acid_profile, pH_base_malt=pH_base_malt, k_RA_to_pH=k_RA_to_pH)

    s = score(acid_profile, target, pH_pred, pH_target, salts_g)
    if (best is None) or (s < best["score"]):
        best = {
            "score": s,
            "fractions": frac,
            "profile": acid_profile,
            "salts_g": salts_g,
            "acid_mL": acid_mL,
            "pH_pred": pH_pred,
            "RA": residual_alkalinity_CaCO3(acid_profile),
        }

best["score"], best["pH_pred"], best["RA"]


In [ ]:
# --- Resultaatrapport ---
fractions = best["fractions"]
profile = best["profile"]
salts_g = best["salts_g"]
acid_mL = best["acid_mL"]

blend_table = pd.DataFrame({
    "Bron": source_names,
    "Fractie": fractions,
    "Volume (L)": fractions*V_total_L
})

ions_table = pd.DataFrame({
    "Ion (mg/L)": keys,
    "Doel": [target[k] for k in keys],
    "Resultaat": [profile[k] for k in keys],
    "Verschil": [profile[k]-target[k] for k in keys],
})

salts_table = pd.DataFrame({
    "Zout": list(salts_g.keys()),
    "Gram per batch": [salts_g[k] for k in salts_g.keys()]
}).sort_values("Gram per batch", ascending=False)

print(f"Stijl: {style}")
print(f"Acid type: {acid_type}")
print(f"Zuur: {acid_mL:.1f} mL per batch")
print(f"Voorspelde mash pH: {best['pH_pred']:.2f} (doel {pH_target:.2f})")
print(f"Residual Alkalinity: {best['RA']:.1f} mg/L as CaCO3")

blend_table, salts_table, ions_table


In [ ]:
# --- Plot: doel vs resultaat ---
x = np.arange(len(keys))
width = 0.35

plt.figure(figsize=(10,4))
plt.bar(x - width/2, [target[k] for k in keys], width, label="Doel")
plt.bar(x + width/2, [profile[k] for k in keys], width, label="Resultaat")
plt.xticks(x, keys)
plt.ylabel("mg/L")
plt.title(f"Waterdesign – {style}")
plt.legend()
plt.tight_layout()
plt.show()

# --- Chloride/Sulfaat verhouding als quick check ---
ratio = profile["Cl"]/max(1e-9, profile["SO4"])
print(f"Cl/SO4 ratio ≈ {ratio:.2f}")


## 5. Interpretatievragen

1. Welke **trade-off** zie je tussen het halen van **SO₄** (IPA) en het beperken van **Ca** binnen realistische grenzen?  
2. Verhoog de weging op pH (**w_pH**) met 2×. Wat gebeurt er met de zoutkeuze en het zuurvolume?  
3. Zet `allowed_sources = ["Tap"]` (dus geen RO). Kan het model nog het Tripel-doel benaderen? Welke ion(en) blokkeren dit?  
4. Voor Stout: verhoog `target["HCO3"]` met +50 mg/L. Welke zoutkeuze neemt toe? Waarom?  
5. Welke aannames in dit notebook zijn het meest “gevaarlijk” in echte brouwwaterchemie (noem er minstens 3)?
